# Bigraph-Builder Demo

In [ ]:
from biosimulator_processes.biosimulator_builder import BiosimulatorBuilder 

## Initialize the builder

In [ ]:
b = BiosimulatorBuilder()

In [ ]:
print(b.list_processes())

In [ ]:
def add_single_process():
    process_type = input(f'Please enter one of the following process types that you wish to add:\n{b.list_processes()}\n:')
    builder_node_name = input('Please enter the name that you wish to assign to this process: ')
    process_kwargs = input('Please enter the process configuration keyword arguments')
    process_args = process_kwargs.split(',')
    input_kwargs = {}
        for arg in process_args:
            proc_args = arg.split('=')
            input_kwargs[proc_args[0]] = proc_args[1]
    visualize = input('Do you wish to visualize this addition after (y/N): ')
    b[builder_node_name].add_process(name=process_type, **dict(process_kwargs))
    b.connect_all()
    if 'N' in visualize:
        b.visualize();
    

In [ ]:
add_single_process()

In [ ]:


input_kwargs


In [ ]:
b['event_process'].add_process(
    name='GillespieEvent',
    kdeg=1.0,  # kwargs fill parameters in the config
)

In [ ]:
# visualize shows the process with its disconnected ports
b.visualize()

### print ports

In [ ]:
b['event_process'].interface(True)

### connect ports using connect_all
`Builder.connect_all` connects ports to stores of the same name.

In [ ]:
b.connect_all(append_to_store_name='_store')
b

In [ ]:
b.visualize()

### add interval process to the config

In [ ]:
b['interval_process'].add_process(
    name='GillespieInterval',
)

In [ ]:
b.visualize()

### connect port to specific target

In [ ]:
# to connect a port in a more targeted way, use connect and specify the port and its target path
b['interval_process'].connect(port='interval', target=['event_process', 'interval']) 

# the remaining ports can connect_all
b.connect_all() 

In [ ]:
b.visualize()

### check current Builder config

In [ ]:
b

## Update the initial state

In [ ]:
initial_state = {
    'DNA_store': {
        'A gene': 2.0,
        'B gene': 1.0},
}
b.update(initial_state)

## Generate composite from builder config and simulate

In [ ]:
composite = b.generate()
composite.run(10)

In [ ]:
composite

## Retrieve the composite document

In [ ]:
doc = b.document()
doc

### save the document to file

In [ ]:
b.write(filename='toy_bigraph')

### load a composite from document
This document represents the full state of the composite, and so can reproduce the previous composite when loaded into a fresh Builder

In [ ]:
b2 = Builder(core=core, file_path='out/toy_bigraph.json')
b2

In [ ]:
b2.visualize()